In [1]:
import json
import numpy as np
import wowsims

spell_id_lists = {
        '0' : "Melee",
        '23881' : "Bloodthirst",
        '1680' : "Whirlwind",
        '47475' : "Slam",
        '47450' : "Heroic Strike",
        '47471' : "Execute",
        '12867': "Deep Wounds",
        '12292' : "Death Wish",
        '1719' : "Recklessness",
        '64382' : "Shattering Throw",
        '54758' : "Engi Gloves",
        '2457' : "Battle Stance",
        '2458' : "Berserker Stance",
        '2687' : 'Bloodrage',
        '2825' : 'Bloodlust',
        '47465': 'Rend',
        '7384': 'Overpower',
        '44949': 'Whirlwind OH',
}

# Load Fury Raid Sim Request JSON

In [2]:
f = open('data/fury-human-bis-p3.json')
settings = json.load(f)

# Set environment and settings

In [3]:
from fury import *

def reset():
    wowsims.new(json.dumps(settings).encode('utf-8'))


# Iterations are currently capped at 3000
iterations = settings['simOptions']['iterations']

duration = settings['encounter']['duration']
reset()

# Default sim agent
Default sim agent is the hardcoded agent from the sim

In [4]:
settings['simOptions']['interactive'] = False
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

c_char_p(None)
Average DPS: 11782.327334307121


In [11]:
settings['simOptions']['interactive'] = False

reset()
while not wowsims.step():
    pass
    
    
cast_metrics = wowsims.getSpellMetrics()

for spell_id, metrics in cast_metrics.items():
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Melee: [1, 1, 1, 1, 4, 62]
Death Wish: [1]
Deep Wounds: [114]
Whirlwind: [12]
Recklessness: [1]
Bloodthirst: [26]
Battle Stance: [8]
Berserker Stance: [8]
Bloodrage: [3]
Bloodlust: [1]
Whirlwind OH: [12]
Heroic Strike: [58]
Rend: [4]
Execute: [5]
Slam: [17]
Engi Gloves: [2]
Shattering Throw: [1]
Overpower: [4]


# PPO Agent

## Load Trained PPO Model

In [8]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.registry import register_env
from fury_sim_env import FurySimEnv
from gymnasium import make
from gymnasium.envs.registration import register

def env_creator(env_config):
    return FurySimEnv(...)

register_env("FurySimEnv", env_creator)

config = PPOConfig()\
            .rollouts(num_rollout_workers=1)

algorithm = config.build(env="FurySimEnv")
algorithm.restore("models\dps-reward_punish-error\PPO\PPO_FurySimEnv_d5a3b_00000_0_2023-05-02_15-25-02\checkpoint_002266")

register(id="FurySimEnv", entry_point="fury_sim_env:FurySimEnv")
env = make("FurySimEnv")

<>:16: DeprecationWarning: invalid escape sequence \d
(RolloutWorker pid=32944) 2023-05-02 20:48:39,727	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
2023-05-02 20:48:39,763	WARNING util.py:67 -- Install gputil for GPU system monitoring.
2023-05-02 20:48:39,812	INFO trainable.py:913 -- Restored on 127.0.0.1 from checkpoint: models\dps-reward_punish-error\PPO\PPO_FurySimEnv_d5a3b_00000_0_2023-05-02_15-25-02\checkpoint_002266
2023-05-02 20:48:39,812	INFO trainable.py:922 -- Current state after restoring: {'_iteration': 2266, '_timesteps_total': None, '_time_total': 17449.62928199768, '_episodes_total': 50009}
c:\ProgramData\Anaconda3\envs\civ6_rl_rllib\lib\site-packages\gymnasium\envs\registration.py:498: UserWarning: WARN: Overriding environment FurySimEnv already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


## Run PPO Model

In [9]:
for i in range(1000):
    observation, info = env.reset()
    terminated = False
    reward = 0
    batch = []
    dps_results = np.array([])
    metrics_batch = []
    debug_logs = []
    debug_logs_batch = []
    while not terminated:
        action = algorithm.compute_single_action(observation)
        observation, reward, terminated, truncated, info = env.step(action)
        batch.append(np.concatenate((observation, [action], [reward])))
        debug_logs.append(info['debug log'])
    debug_logs_batch.append(debug_logs)
    dps_results = np.append(dps_results, info['dps'])
    metrics_batch.append(info['spell metrics'])

In [10]:
median = np.argsort(dps_results)[len(dps_results)//2]
print(f'Average DPS: {dps_results[median]}')

cast_metrics = metrics_batch[median]
for spell_id, metrics in cast_metrics.items():
    # Only one target, so we can just take the first one
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Average DPS: 9576.87978273188
Melee: [1, 17, 62]
Death Wish: [1]
Deep Wounds: [99]
Whirlwind: [7]
Recklessness: [1]
Bloodthirst: [28]
Bloodrage: [3]
Bloodlust: [1]
Whirlwind OH: [7]
Heroic Strike: [44]
Slam: [3]
Engi Gloves: [2]
Shattering Throw: [1]


### Debug Log

In [11]:
actions_id = {
    0: "Bloodthirst", 
    1: "Whirlwind",
    2: "Slam",
    3: "HeroicStrike",
    4: "Execute",
    5: "DeathWish",
    6: "Recklessness",
    7: "ShatteringThrow",
    8: "Bloodrage",
    9: "EngiGlove",
    10: "Bloodlust",
}

In [12]:
import pandas as pd

df = pd.DataFrame(debug_logs_batch[median], columns=['Timestamp', 'Action', 'Successful Cast', 'Damage Done', 'Total Damage Done', 'Rage'])
df['Action'] = df['Action'].map(actions_id)
df['DPS'] = df['Total Damage Done'].div(df['Timestamp'])
df

,Timestamp,Action,Successful Cast,Damage Done,Total Damage Done,Rage,DPS
0,0.000000,EngiGlove,True,2982.469655,2.982470e+03,30.972979,inf
1,0.000000,Bloodlust,True,0.000000,2.982470e+03,30.972979,inf
2,0.000000,DeathWish,True,0.000000,2.982470e+03,20.972979,inf
3,0.010000,Recklessness,True,0.000000,2.982470e+03,20.972979,2.982470e+05
4,0.705782,ShatteringThrow,True,6400.673194,9.383143e+03,86.523617,1.329467e+04
...,...,...,...,...,...,...,...
469,115.837786,Recklessness,False,2950.091206,1.119182e+06,84.158064,9.661633e+03
470,116.246931,Bloodthirst,False,1838.809077,1.121021e+06,84.158064,9.643446e+03
471,116.892613,ShatteringThrow,False,0.000000,1.121021e+06,84.158064,9.590178e+03
472,117.246931,DeathWish,False,1838.809077,1.122860e+06,84.158064,9.576880e+03


In [13]:
df.to_csv("debug_logs/debug_log_dps_reward-punish_error.csv")

# Spell Debug

In [4]:
reset()
Spells.register()
Auras.register()
TargetAuras.register()

settings['simOptions']['interactive'] = True
damages = np.array([])
spell_metrics = []

for i in range(1):
    reset()
    while not wowsims.step():
        if wowsims.needsInput():
            wowsims.trySpell(Spells.HeroicStrike)
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)
    spell_metrics.append(wowsims.getSpellMetrics())

median = np.argsort(damages)[len(damages)//2]
print(f'Average DPS: {damages[median] / duration}')

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [7]:
settings['simOptions']['interactive'] = True

reset()
while not wowsims.step():
    pass
cast_metrics = spell_metrics[median]
for spell_id, metrics in cast_metrics.items():
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Melee: [1, 54]
Deep Wounds: [67]
Heroic Strike: [54]
